In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from pyrcn.echo_state_network import ESNRegressor

structural_mask = pd.read_csv('data/sub-CON01_ses-preop.csv',header=None)
structural_mask = (structural_mask > 0).astype(int)
# sns.heatmap(structural_mask)

In [2]:
df = pd.read_csv('data/sub-CON01_ses-preop_task-rest_space-MNI152NLin2009cAsym_atlas-Gordon_desc-timeseries_bold.tsv', header=None)
df = df.T

In [3]:
x = df.iloc[111]
y = df.iloc[222]
x = x.values.reshape(-1,1)
y = y.values.reshape(-1,1)

In [4]:
train_len = int(len(x) * 0.6)

X_train = np.array(x[:train_len]).reshape(-1, 1)
X_test = np.array(x[train_len:]).reshape(-1, 1)
y_train = np.array(y[:train_len])
y_test = np.array(y[train_len:])

In [25]:
reg = ESNRegressor(hidden_layer_size=30)

In [26]:
reg.fit(X=X_train, y=y_train)

ESNRegressor(input_to_node=InputToNode(hidden_layer_size=30),
             node_to_node=NodeToNode(hidden_layer_size=30),
             regressor=IncrementalRegression(), requires_sequence=False)

In [27]:
y_pred = reg.predict(X_test)


In [28]:
# correlation
np.corrcoef(y_test.reshape(-1), y_pred.reshape(-1))[0, 1]

0.2631532495631058

In [29]:
# plt.figure(figsize=(20, 10))
# plt.plot(y_test, label='y_test')
# plt.plot(y_pred, label='y_pred')

In [30]:
# correlation on trianing set
y_pred_train = reg.predict(X_train)
np.corrcoef(y_train.reshape(-1), y_pred_train.reshape(-1))[0, 1]

0.6803766033527329